In [1]:
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import pandas as pd
import folium
import requests
import json
import webbrowser

In [2]:
ORS_API_KEY = '5b3ce3597851110001cf6248943b6a36013a42f59fa7b6d69967ea94'

In [3]:
# places to visit enter the Lat,Lon need to route
place = [['0',35.69733964,-88.37940135],
         ['1',35.70387548,-88.27184805], 
         ['2',35.68062382,-88.25015472],
         ['3',35.690187,-88.23135383],
         ['4',35.67708356,-88.39348067],
         ]

In [4]:
# Create pandas dataframe
df = pd.DataFrame(place, columns = ['place', 'Lat','Lon'])
df

,place,Lat,Lon
0,0,35.697340,-88.379401
1,1,35.703875,-88.271848
2,2,35.680624,-88.250155
3,3,35.690187,-88.231354
4,4,35.677084,-88.393481


In [5]:
# tip: https://openrouteservice.org/example-optimize-pub-crawl-with-ors/
# Mapping using folium
my_map = folium.Map(
    location=[35.697340, -88.379401],
    tiles='Stamen Toner',
    zoom_start=13)
df.apply(lambda row:folium.Marker(location=[row['Lat'], row['Lon']]).add_to(my_map), axis=1)

# display map
my_map

In [6]:
# Data for finding shortest route
start_location = [df.iloc[0]['Lon'], df.iloc[0]['Lat']]
end_location = [df.iloc[0]['Lon'], df.iloc[0]['Lat']]
vehicle = [{ "id":1,
            "profile":"driving-car",
            "start":start_location,
            "end":end_location,
            "capacity":[6]    
}]
# stops
stops = []
for index, row in df[1:].iterrows():
    stop = {}
    stop['id'] = index + 1
    stop['service'] = 300
    stop['delivery'] = [1]
    stop['location'] =  [row['Lon'], row['Lat']]
    stops.append(stop) 

In [7]:
stops

[{'id': 2,
  'service': 300,
  'delivery': [1],
  'location': [-88.27184805, 35.70387548]},
 {'id': 3,
  'service': 300,
  'delivery': [1],
  'location': [-88.25015472, 35.68062382]},
 {'id': 4,
  'service': 300,
  'delivery': [1],
  'location': [-88.23135383, 35.690187]},
 {'id': 5,
  'service': 300,
  'delivery': [1],
  'location': [-88.39348067, 35.67708356]}]

In [8]:
# api call to openrouteservice

body = {'jobs':stops, 'vehicles': vehicle}
headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Authorization': ORS_API_KEY,
    'Content-Type': 'application/json; charset=utf-8'
}
response = requests.post('https://api.openrouteservice.org/optimization', json=body, headers=headers)

print(response.status_code, response.reason)
print(response.text)

200 OK
{"code":0,"summary":{"cost":5001,"unassigned":0,"delivery":[4],"amount":[4],"pickup":[0],"service":1200,"duration":5001,"waiting_time":0,"computing_times":{"loading":61,"solving":1}},"unassigned":[],"routes":[{"vehicle":1,"cost":5001,"delivery":[4],"amount":[4],"pickup":[0],"service":1200,"duration":5001,"waiting_time":0,"steps":[{"type":"start","location":[-88.37940135,35.69733964],"load":[4],"arrival":0,"duration":0},{"type":"job","location":[-88.23135383,35.690187],"id":4,"service":300,"waiting_time":0,"job":4,"load":[3],"arrival":1940,"duration":1940},{"type":"job","location":[-88.25015472,35.68062382],"id":3,"service":300,"waiting_time":0,"job":3,"load":[2],"arrival":2823,"duration":2523},{"type":"job","location":[-88.27184805,35.70387548],"id":2,"service":300,"waiting_time":0,"job":2,"load":[1],"arrival":3413,"duration":2813},{"type":"job","location":[-88.39348067,35.67708356],"id":5,"service":300,"waiting_time":0,"job":5,"load":[0],"arrival":5360,"duration":4460},{"type":

In [9]:
# Total Drive Time in minutes
data = response.json()
total_drive_time = data['summary']['cost'] / 60
# Shortest Route
steps = data['routes'][0]['steps']
route_points = []
for step in steps:
    route_step = (step['location'][1], step['location'][0])
    route_points.append(route_step)
route_points

[(35.69733964, -88.37940135),
 (35.690187, -88.23135383),
 (35.68062382, -88.25015472),
 (35.70387548, -88.27184805),
 (35.67708356, -88.39348067),
 (35.69733964, -88.37940135)]

In [10]:
route_pairs = []
for index, item in enumerate(route_points):
    if index < len(route_points)-1:
        route_pairs.append([item, route_points[index+1]])
route_pairs

[[(35.69733964, -88.37940135), (35.690187, -88.23135383)],
 [(35.690187, -88.23135383), (35.68062382, -88.25015472)],
 [(35.68062382, -88.25015472), (35.70387548, -88.27184805)],
 [(35.70387548, -88.27184805), (35.67708356, -88.39348067)],
 [(35.67708356, -88.39348067), (35.69733964, -88.37940135)]]

In [11]:


def get_driving_route(source_coordinates, dest_coordinates):
    parameters = {
    'api_key': ORS_API_KEY,
    'start' : '{},{}'.format(source_coordinates[1], source_coordinates[0]),
    'end' : '{},{}'.format(dest_coordinates[1], dest_coordinates[0])
    }

    response = requests.get(
        'https://api.openrouteservice.org/v2/directions/driving-car', params=parameters)

    if response.status_code == 200:
        data = response.json()
    else:
        print('Request failed.')
    

    def swap(coord):
        coord[0],coord[1]=coord[1],coord[0]
        return coord

    summary = data['features'][0]['properties']['summary']
    route= data['features'][0]['geometry']['coordinates']
    
    route=list(map(swap, route))
    return route
    
routes = []
for source, destination in route_pairs:
    route = get_driving_route(source, destination)
    routes.append(route)


In [12]:
# tip: https://deparkes.co.uk/2016/06/03/plot-lines-in-folium/
# Mapping using folium

ave_lat = sum(p[0] for p in route_points)/len(route_points)
ave_lon = sum(p[1] for p in route_points)/len(route_points)
 
# Load map centred on average coordinates
route_map = folium.Map(
    location=[ave_lat, ave_lon],
    tiles='Stamen Toner',
    zoom_start=13)
 
#add a markers

folium.Marker(route_points[0], icon=folium.Icon(color='green'), tooltip='Start/End').add_to(route_map)

for index, each in enumerate(route_points[1:-1]):
    folium.Marker(each, icon=folium.Icon(color='blue'), tooltip=str(index+1)).add_to(route_map)


for route in routes:
    folium.PolyLine(
        route,
        weight=8,
        color='blue',
        opacity=0.6,
        ).add_to(route_map)
route_map

In [13]:
route_map.save('route.html')